## **Trigram-based Markov Model for Name Generation**

### Introduction

This notebook implements a Trigram-based Markov Model for name generation. The goal is to learn patterns from a dataset of names and use those patterns to generate new, plausible names.

Specifically, the model learns probabilities based on two-letter sequences ("bigrams") and predicts the next character. To avoid zero-probability issues for unseen sequences, Laplace smoothing is applied.

#### Step 1: Load and Process Dataset

Load all names from the dataset `names.txt` into a list, as this allows us to process and analyze the data efficiently.

In [1]:
with open('../data/names.txt', 'r') as f:
    names = f.read().splitlines()
print(len(names))

32033


#### Step 2: Building the Bigram-to-Character Probability Table



We create a nested dictionary `bi_pairs` to store the occurrences of each possible character following every bigram in our dataset.

We also apply Laplace smoothing to make sure that all combinations have a **non-zero** probability.

In [2]:
from collections import defaultdict

bi_pairs = defaultdict(lambda: defaultdict(int))

chs = {ch for name in names for ch in name}
chs.add('.')
chs = sorted(chs)

# apply Laplace smoothing
for ch1 in chs:
    for ch2 in chs:
        for ch3 in chs:
            bi_pairs[ch1+ch2][ch3] += 1

# process names into bigrams
for name in names:
    name = f'.{name}.'
    for ch1, ch2, ch3 in zip(name, name[1:], name[2:]):
        pair = ch1 + ch2 # Pair up first character and second character
        bi_pairs[pair][ch3] += 1 # increment the value of the occurences of the character 3 after our pair

In [3]:
# For debugging

outs_of_gh = sorted(bi_pairs['gh'].items(), key=lambda x:x[1], reverse=True) # e.g. raylei`gh`, to see if our pairs are fine.
# usually the sequence `gh` ends with a `.` in names so `.` should be the most common
assert len(outs_of_gh) == 27 and len(bi_pairs) == 27**2, \
    'Shape mismatch. All bigrams and character pairs must be defined.'
print("Shape OK")

print([out for out in outs_of_gh if out[1] > 1])

Shape OK
[('.', 236), ('a', 59), ('t', 33), ('n', 16), ('l', 8), ('e', 6), ('i', 5), ('b', 3), ('o', 3)]


#### Step 3: Encoding Bigram and Output Characters
We encode each unique bigram and character into integer IDs. This numeric encoding simplifies operations like tensor manipulations and sampling.

> **NB:** The reasoning behind using a generator for ID assignment instead of using dictionary length is because:
>
> * It is much faster for larger datasets;
>
> * If we were to delete keys in the future, our ID assignments wouldn't shift.

In [4]:
from itertools import count

bigram_id_gen = count() # Generate IDs for each encoding
output_id_gen = count()

bigram_ids = {} # ID : bigram
output_ids = {} # ID : output

for ch in chs: # Encode characters into integers
    output_ids[next(output_id_gen)] = ch

for bigram in bi_pairs.items(): # also encode each bigram into an integer
    bigram_ids[next(bigram_id_gen)] = bigram[0]

#### Step 4: Convert Counts into Probabilities

Next, we convert the frequency counts into probabilities.

For each bigram, we create a probability distribution over the characters that follow it.

In [5]:
bi_pairs_prob = defaultdict(lambda: defaultdict(int))

for bigram, outs in bi_pairs.items():
    for chr, count in outs.items():
        bi_pairs_prob[bigram][chr] = count / sum(outs.values())

#### Step 5: Sampling Names from the Model

Using the probability distributions, we generate new names character-by-character.

We start with a provided initial character and continue sampling until we reach the end token `.`.

In [6]:
import torch

In [7]:
while True:
    word = ''
    chr = input("Enter first character of the name: ").lower()
    if chr not in chs:
        print("--------\nGoodbye!")
        break
    chr = '.' + chr[-1]
    word = f'{chr[-1]}'
    
    while True:
        chr_tensor = torch.tensor(list(bi_pairs_prob[chr].values()))
        sample = torch.multinomial(chr_tensor, 1, replacement=True)
        chr = chr[-1] + output_ids[sample.item()]
        if chr[-1] == '.':
            break
        word += chr[-1]
    print(f"Name starting with {word[0]}: {word}")


Enter first character of the name:  a


Name starting with a: ailah


Enter first character of the name:  b


Name starting with b: brrna


Enter first character of the name:  c


...

Name starting with d: dileia


Enter first character of the name:  e


Name starting with e: em


Enter first character of the name:  1337


--------
Goodbye!
